### Hello World with KNative

In [1]:
%cd /Users/i033085/externalRepos/docs/serving/samples/helloworld-python


/Users/i033085/externalRepos/docs/serving/samples/helloworld-python


In [2]:
! kubectl apply -f service.yaml

service.serving.knative.dev "helloworld-python" created


In [3]:
! kubectl get svc istio-ingressgateway --namespace istio-system

NAME                   TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)                                                                                                                   AGE
istio-ingressgateway   NodePort   10.108.14.171   <none>        80:31380/TCP,443:31390/TCP,31400:31400/TCP,15011:30638/TCP,8060:31749/TCP,853:31693/TCP,15030:31520/TCP,15031:31924/TCP   34d


In [4]:
! kubectl get ksvc helloworld-python  --output=custom-columns=NAME:.metadata.name,DOMAIN:.status.domain

NAME                DOMAIN
helloworld-python   helloworld-python.default.example.com


In [3]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v2!


In [6]:
! kubectl -n knative-serving get cm config-autoscaler --output json

{
    "apiVersion": "v1",
    "data": {
        "_example": "################################\n#                              #\n#    EXAMPLE CONFIGURATION     #\n#                              #\n################################\n\n# This block is not actually functional configuration,\n# but serves to illustrate the available configuration\n# options and document them in a way that is accessible\n# to users that `kubectl edit` this config map.\n#\n# These sample configuration options may be copied out of\n# this block and unindented to actually change the configuration.\n\n# The Revision ContainerConcurrency field specifies the maximum number\n# of requests the Container can handle at once. Container concurrency\n# target percentage is how much of that maximum to use in a stable\n# state. E.g. if a Revision specifies ContainerConcurrency of 10, then\n# the Autoscaler will try to maintain 7 concurrent connections per pod\n# on average. A value of 0.7 is chosen because the Autoscaler p

### Role out a new Revision

In [4]:
! kubectl apply -f service-rev1.yaml

service.serving.knative.dev "helloworld-python" configured


In [5]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v3!


In [7]:
! kubectl get revisions

NAME                                 AGE
helloworld-python-8hb66              10m
helloworld-python-vxdl8              1m
mlf-python-grpc-model-server-72rl5   12h


In [8]:
! kubectl get route --output yaml

apiVersion: v1
items:
- apiVersion: serving.knative.dev/v1alpha1
  kind: Route
  metadata:
    creationTimestamp: 2019-03-26T18:15:21Z
    finalizers:
    - routes.serving.knative.dev
    generation: 1
    labels:
      serving.knative.dev/service: helloworld-python
    name: helloworld-python
    namespace: default
    ownerReferences:
    - apiVersion: serving.knative.dev/v1alpha1
      blockOwnerDeletion: true
      controller: true
      kind: Service
      name: helloworld-python
      uid: 1e098bc7-4ff3-11e9-a36e-025000000001
    resourceVersion: "867358"
    selfLink: /apis/serving.knative.dev/v1alpha1/namespaces/default/routes/helloworld-python
    uid: 1e0e0bd9-4ff3-11e9-a36e-025000000001
  spec:
    traffic:
    - configurationName: helloworld-python
      percent: 100
  status:
    address:
      hostname: helloworld-python.default.svc.cluster.local
    conditions:
    - lastTransitionTime: 2019-03-26T18:15:27Z
      status: "True"
      type: AllTrafficAssigned
    - lastTr

In [9]:
! kubectl apply -f service-rev2.yaml

service.serving.knative.dev "helloworld-python" configured


In [10]:
! kubectl get route --output yaml

apiVersion: v1
items:
- apiVersion: serving.knative.dev/v1alpha1
  kind: Route
  metadata:
    creationTimestamp: 2019-03-26T18:15:21Z
    finalizers:
    - routes.serving.knative.dev
    generation: 2
    labels:
      serving.knative.dev/service: helloworld-python
    name: helloworld-python
    namespace: default
    ownerReferences:
    - apiVersion: serving.knative.dev/v1alpha1
      blockOwnerDeletion: true
      controller: true
      kind: Service
      name: helloworld-python
      uid: 1e098bc7-4ff3-11e9-a36e-025000000001
    resourceVersion: "867514"
    selfLink: /apis/serving.knative.dev/v1alpha1/namespaces/default/routes/helloworld-python
    uid: 1e0e0bd9-4ff3-11e9-a36e-025000000001
  spec:
    traffic:
    - name: current
      percent: 50
      revisionName: helloworld-python-8hb66
    - name: candidate
      percent: 50
      revisionName: helloworld-python-vxdl8
    - configurationName: helloworld-python
      name: latest
      percent: 0
  status:
    address:
    

In [15]:
! curl -H "Host: helloworld-python.default.example.com" http://localhost:31380

Hello Python Sample v3!


In [40]:
! hey -z 30s -c 50 \
  -host "helloworld-python.default.example.com" \
  "http://localhost:31380?sleep=100" \

panic: runtime error: invalid memory address or nil pointer dereference
[signal SIGSEGV: segmentation violation code=0x1 addr=0x0 pc=0x1556e89]

goroutine 1 [running]:
main.main()
	/Users/i033085/golang/src/github.com/sbcd90/hey/hey.go:244 +0x4e9


In [34]:
! kubectl delete -f service.yaml

service.serving.knative.dev "helloworld-python" deleted


### Auto Scaling

In [ ]:
%cd /Users/i033085/externalRepos/docs/serving/samples/autoscale-go

In [ ]:
! kubectl apply -f service.yaml

In [ ]:
! curl --header "Host: autoscale-go.default.example.com" "http://localhost:31380?prime=10000&bloat=5"

In [ ]:
! hey -z 30s -c 50 \
  -host "autoscale-go.default.example.com" \
  "http://localhost:31380?prime=10000&bloat=5" \
  && kubectl get pods

In [ ]:
! kubectl delete -f service.yaml

In [ ]:
! kubectl get pods

### Deploy gRPC model Server on KNative

In [17]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/knative

/Users/i033085/gitrepos/mlf-python-grpc-model-server/knative


In [18]:
!kubectl apply -f sample.yaml

service.serving.knative.dev "mlf-python-grpc-model-server" configured


In [19]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/test

/Users/i033085/gitrepos/mlf-python-grpc-model-server/test


In [20]:
!python python_grpc_client.py 


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

outputs {
  key: "col"
  value {
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: 1
      }
    }
    string_val: "60.400001525878906"
  }
}



In [21]:
!hey -n 10000 -c 100 -M tensorflow-feature-sum-model -v 1 -k X -d 1,2,3,4 -server-host-override mlf-python-grpc-model-server.default.example.com -insecure -g localhost:31380


Summary:
  Total:	101.8819 secs
  Slowest:	9.4073 secs
  Fastest:	0.0122 secs
  Average:	0.9987 secs
  Requests/sec:	98.1529
  
  Total data:	210000 bytes
  Size/request:	21 bytes

Response time histogram:
  0.012 [1]	|
  0.952 [7074]	|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
  1.891 [2289]	|■■■■■■■■■■■■■
  2.831 [186]	|■
  3.770 [105]	|■
  4.710 [236]	|■
  5.649 [10]	|
  6.589 [15]	|
  7.528 [27]	|
  8.468 [31]	|
  9.407 [26]	|


Latency distribution:
  10% in 0.4638 secs
  25% in 0.6133 secs
  50% in 0.7821 secs
  75% in 1.0038 secs
  90% in 1.3450 secs
  95% in 2.6398 secs
  99% in 5.4958 secs

Details (average, fastest, slowest):
  DNS+dialup:	0.0127 secs, 0.0122 secs, 9.4073 secs
  DNS-lookup:	0.0000 secs, 0.0000 secs, 0.0000 secs
  req write:	0.0000 secs, 0.0000 secs, 0.0000 secs
  resp wait:	0.0000 secs, 0.0000 secs, 0.0000 secs
  resp read:	0.0000 secs, 0.0000 secs, 0.0008 secs

Status code distribution:
  [200]	10000 responses





In [ ]:
%cd /Users/i033085/gitrepos/mlf-python-grpc-model-server/knative

In [ ]:
! kubectl delete -f sample.yaml